<a href="https://colab.research.google.com/github/sathvik3103/Pyspark/blob/main/Basics_of_Pyspark_MLib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pyspark
import pyspark as pk
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Practice').getOrCreate()

In [ ]:
spark

In [ ]:
df_pyspark=spark.read.csv('/Tailwind Traders Sales.csv',header=True,inferSchema=True)
df_pyspark.show()

+---------+-----------------+--------------------+--------------------+-------------------+---------------+------------------+---+-----+------+----------------+------------+--------+--------+--------+-------+--------+-----------+-----------+-----------+--------------------+---------+----------+------------------+---------------+
| OrderID | Customer Name   |        Product Name| Product Description|Gross Product Price|Tax Per Product|Quantity Purchased| GR|   TT|    NR|Product Category| SKU Number | Weight | Color  |  Size  | Rating| Stock  | Country ID| Sales Rep | CustomerID|     Address        | Zip Code| Phone    | Email            | Loyalty Points|
+---------+-----------------+--------------------+--------------------+-------------------+---------------+------------------+---+-----+------+----------------+------------+--------+--------+--------+-------+--------+-----------+-----------+-----------+--------------------+---------+----------+------------------+---------------+
|     1

In [ ]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=[" Country ID", "Gross Product Price"],outputCol="Independent Features")
output=featureassembler.transform(df_pyspark)
output.show()

+---------+-----------------+--------------------+--------------------+-------------------+---------------+------------------+---+-----+------+----------------+------------+--------+--------+--------+-------+--------+-----------+-----------+-----------+--------------------+---------+----------+------------------+---------------+--------------------+
| OrderID | Customer Name   |        Product Name| Product Description|Gross Product Price|Tax Per Product|Quantity Purchased| GR|   TT|    NR|Product Category| SKU Number | Weight | Color  |  Size  | Rating| Stock  | Country ID| Sales Rep | CustomerID|     Address        | Zip Code| Phone    | Email            | Loyalty Points|Independent Features|
+---------+-----------------+--------------------+--------------------+-------------------+---------------+------------------+---+-----+------+----------------+------------+--------+--------+--------+-------+--------+-----------+-----------+-----------+--------------------+---------+----------+-

In [ ]:
final_df = output.select("Independent Features","Tax Per Product")
final_df.show()

+--------------------+---------------+
|Independent Features|Tax Per Product|
+--------------------+---------------+
|          [1.0,60.0]|            8.4|
|          [5.0,35.0]|           2.45|
|         [1.0,250.0]|           17.5|
|          [2.0,15.0]|           1.05|
|          [1.0,55.0]|           3.85|
|          [2.0,75.0]|           5.25|
|          [3.0,50.0]|           14.7|
|          [2.0,90.0]|            6.3|
|         [1.0,180.0]|           12.6|
|           [1.0,3.0]|            1.4|
|         [2.0,300.0]|           21.0|
|          [4.0,50.0]|            3.5|
|          [1.0,60.0]|            4.2|
|          [5.0,40.0]|            2.8|
|          [5.0,45.0]|           3.15|
|          [1.0,80.0]|            5.6|
|         [5.0,200.0]|           14.0|
|          [5.0,65.0]|           4.55|
|         [1.0,110.0]|            7.7|
|          [2.0,95.0]|           6.65|
+--------------------+---------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=final_df.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Tax Per Product')
regressor=regressor.fit(train_data)

In [ ]:
regressor.coefficients

DenseVector([-0.1861, 0.0654])

In [ ]:
regressor.intercept

1.4302394058965116

In [ ]:
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+---------------+------------------+
|Independent Features|Tax Per Product|        prediction|
+--------------------+---------------+------------------+
|          [1.0,60.0]|            4.2|  5.16770047600594|
|          [1.0,80.0]|            5.6| 6.475558464408023|
|         [1.0,120.0]|            8.4| 9.091274441212187|
|         [1.0,210.0]|           14.7|14.976635389021558|
|         [1.0,250.0]|           17.5|17.592351365825724|
|          [2.0,18.0]|           1.26| 2.235085805264748|
|          [2.0,48.0]|           3.36| 4.196872787867872|
|         [2.0,205.0]|          14.35| 14.46355799682422|
|         [2.0,235.0]|          16.45|16.425344979427344|
|          [3.0,63.0]|           4.41| 4.991653384072615|
|          [4.0,46.0]|           3.22|3.6938611988340257|
|          [4.0,85.0]|           5.95| 6.244184276218086|
|          [5.0,22.0]|           1.54|1.9383187176547085|
|          [5.0,40.0]|            2.8|3.1153909072165824|
|          [5.

In [ ]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError,pred_results.r2,pred_results.rootMeanSquaredError

(0.4485721192420498, 0.3122382537195496, 0.991280323354654, 0.5587828323414649)